In [1]:
name = "CombineSignals"
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
alphas = ["UserItemBiases", "MatrixFactorization", "ItemCF", "ItemEmbedCF"];

In [4]:
function evaluate(X, y, β)
    y_pred = X * β
    rmse(y, y_pred), mae(y, y_pred), r2(y, y_pred)
end;

In [5]:
function get_dep(split)
    return get_split(split).rating
end

function get_indep(split, alphas)
    users = get_split(split).user
    X = zeros(length(users), length(alphas))
    for j = 1:length(alphas)
        X[:, j] = get_alpha(alphas[j], split).rating
    end
    X
end;

In [6]:
β = get_indep("validation", alphas) \ get_dep("validation")

4-element Vector{Float64}:
 0.9939583729002062
 0.38023606878549543
 0.49325456733704615
 0.2642719012058097

In [7]:
evaluate(get_indep("validation", alphas), get_dep("validation"), β)

(1.1271179449743902, 0.8239820342739057, 0.5993156215547039)

In [8]:
evaluate(get_indep("test", alphas), get_dep("test"), β)

(1.1274617709707389, 0.8243731412856492, 0.5991431195985182)

## TODO store Coefs